In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)


# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os


import warnings
warnings.filterwarnings("ignore")

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb

In [ ]:
print("XGBoost version:", xgb.__version__)

In [ ]:

print('# File sizes')
total_size = 0
start_path = '../input/jane-street-market-prediction'  # To get size of current directory
for path, dirs, files in os.walk(start_path):
    for f in files:
        fp = os.path.join(path, f)
        total_size += os.path.getsize(fp)
print("Directory size: " + str(round(total_size/ 1000000, 2)) + 'MB')

In [ ]:

%%time
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

In [ ]:
print('train shape is {}'.format(train.shape))
print('features shape is {}'.format(features.shape))
print('example_test shape is {}'.format(example_test.shape))
print('sample_prediction_df shape is {}'.format(sample_prediction_df.shape))

In [ ]:
train.head()


In [ ]:

missing_values_count = train.isnull().sum()
print (missing_values_count)
total_cells = np.product(train.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

In [ ]:

# I have taked this cell from https://www.kaggle.com/jazivxt/the-market-is-reactive
# And https://www.kaggle.com/drcapa/jane-street-market-prediction-starter-xgb

train = train[train['weight'] != 0]

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')


X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

In [ ]:
def feature_sign(data):
    for f in data.columns[20:40]:
        if 'feature' in f:
            data[f+'_sign'] = (data[f].values > 0).astype('int')
    return data

In [ ]:
X_train = feature_sign(X_train)


In [ ]:
X_train = X_train.fillna(-999)


In [ ]:
x = train['action'].value_counts().index
y = train['action'].value_counts().values

trace2 = go.Bar(
     x=x ,
     y=y,
     marker=dict(
         color=y,
         colorscale = 'Viridis',
         reversescale = True
     ),
     name="Imbalance",    
 )
layout = dict(
     title="Data imbalance - action",
     #width = 900, height = 500,
     xaxis=go.layout.XAxis(
     automargin=True),
     yaxis=dict(
         showgrid=False,
         showline=False,
         showticklabels=True,
 #         domain=[0, 0.85],
     ), 
)
fig1 = go.Figure(data=[trace2], layout=layout)
iplot(fig1)

In [ ]:
del x, y, train


In [ ]:
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=-999,
    random_state=2020,
)

In [ ]:
%time clf.fit(X_train, y_train)


In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    X_test = feature_sign(X_test)
    X_test = X_test.fillna(-999)
    y_preds = clf.predict(X_test)
    sample_prediction_df.action = y_preds
    submission=env.predict(sample_prediction_df)

    

    

In [ ]:
submission

In [ ]:
submission=pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv',sep=',')

submission.to_csv('./submission.csv',index=False)
